# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096874


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:37,  3.37s/it]

Rendering models:  10%|▉         | 3/31 [00:06<01:07,  2.41s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:46,  1.72s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:32,  1.24s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:09<00:00,  7.82it/s]

Rendering models:  94%|█████████▎| 29/31 [00:09<00:00,  6.11it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  5.35it/s]

Rendering models: 100%|██████████| 31/31 [00:10<00:00,  5.75it/s]

matiasd123                             0.000571
not-logged-in-8cf2e6929490167ca6ab     0.000456
not-logged-in-46a23861c6275e310c67     0.000171
jnarayanbvg                            0.000174
Borkdoge123                            0.017402
not-logged-in-c6917d9b19f41e6c2995     0.000364
not-logged-in-c6917d9b19f41e6c2995     0.001196
Hunting_X                              0.007689
not-logged-in-c82b158ee9f8100a6b4e     0.000278
Poupoussin                             0.000455
lsautter                               0.000260
tingard                                0.000162
KjellN                                 0.000138
gavinrider                             0.000823
AdamMorse                              0.000205
Ellaida                                0.001310
41Letters                              0.000184
TRISTANM18                             0.000184
Ellaaa666                              0.001251
Lavadude                               0.003215
Rian_NL                                0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())